In [91]:

# necessary imports
import numpy as np
import csv
import os
from google.colab import drive 
import matplotlib.pyplot as plt
import random # Added by Caleb
import math # Added by Caleb
import os
import torch
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [92]:
from google.colab import drive 
drive.mount('/content/drive/') ## mount to drive. This will ask for permission to access your Google drive each time

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [93]:
data_folder_path = "/content/drive/Shareddrives/Neural Nets/Competition/ECE542_sp2022_Project_TerrainRecognition/" # path into Lobton's directory 
type_of_data = "TestData" # Read in the type of data you want. Options are either:  'TrainingData' or 'TestData'
TRAINING = True
list_of_files = os.listdir(data_folder_path + type_of_data) # List everything in the directory at place 2022_Project_TerrainRecognition/TrainingData or /TestData (from line above)
list_of_files.sort() # Sort the list of files
print(list_of_files) # Print out the list of files
# need to load all this data in for augmentation (only the x data, but need to match what y data it connects with)
# we need to figure out what data we want to use too. 

['subject_009_01__x.csv', 'subject_009_01__x_time.csv', 'subject_009_01__y_time.csv', 'subject_010_01__x.csv', 'subject_010_01__x_time.csv', 'subject_010_01__y_time.csv', 'subject_011_01__x.csv', 'subject_011_01__x_time.csv', 'subject_011_01__y_time.csv', 'subject_012_01__x.csv', 'subject_012_01__x_time.csv', 'subject_012_01__y_time.csv']


In [94]:
class CustomImageDataset(Dataset):
    def __init__(self, x_data, y_data, img_dir=None, transform=None, target_transform=None):
        self.img_labels = y_data
        self.img_dir = img_dir
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.x_data[idx]
        label = self.y_data[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


In [95]:
# Code block created by Caleb
# Goal: Session class that holds subject and session number, and also returns the data for that session

# This class can return the name of the file that holds the requested data
    # Example of how to use:
    # newSession = Session(4, 5)
    # newStr = newSession.xTimeName()
    # print(newStr)
    # # Prints out "subject_004_05__x.csv"

class Session:
  def __init__(self, subject_number, session_number):
    self.subject_number = int(subject_number)  # Initialize subject number
    self.session_number = int(session_number)  # Initialize session number

  # Each of the following member functions return the name of the specified file for that subject and session number of the Session:

  def xTimeName(self):
    return "subject_" + f"{self.subject_number:03d}" + "_" + f"{self.session_number:02d}" + "__x_time.csv"

  def yTimeName(self):
    return "subject_" + f"{self.subject_number:03d}" + "_" + f"{self.session_number:02d}" + "__y_time.csv"

  def xDataName(self):
    return "subject_" + f"{self.subject_number:03d}" + "_" + f"{self.session_number:02d}" + "__x.csv"

  def yDataName(self):
    return "subject_" + f"{self.subject_number:03d}" + "_" + f"{self.session_number:02d}" + "__y.csv"


  # This function input is the Session object that contians the subject and session numbers
  def getXDataFromFile(self):
    x_data_path = data_folder_path + "TestData/" + self.xDataName() # Get the path to the x_data file 
    x_data = np.genfromtxt(x_data_path, delimiter=',')  # Read the data in from the text file
    return x_data  # Return the data array
    # This function returns the array of all the six x values

  def getXTimeFromFile(self):
    x_time_path = data_folder_path + "TestData/" + self.xTimeName() # Get the path to the x_data file 
    x_time = np.genfromtxt(x_time_path, delimiter=',')  # Read the data in from the text file
    return x_time  # Return the data array
    # This function returns the array of just the x time values

  def getYTimeFromFile(self):
    y_time_path = data_folder_path + "TestData/" + self.yTimeName() # Get the path to the x_data file 
    y_time = np.genfromtxt(y_time_path, delimiter=',')  # Read the data in from the text file
    return y_time  # Return the data array
    # This function returns the array of just the y time values




In [96]:
# Code block created by Caleb:
# Goal is to create list of each session, with a unique session id (session id made up of subject number and session number)

list_of_sessions = []

for file_name in list_of_files:
  # Extract subject and session number from file_name:
  new_subject_number = int(file_name[8:11]) # Since all provided subject and session numbers are single digit, we can 
  new_session_number = int(file_name[12:14]) #   pick them out of the file name by grabbing an exact character number.
  


  # Check if I should add this session to the list of sessions
  should_add_session = True # Create a variable that by default should add the session
  for session in list_of_sessions: # Loop through the sessions in the list of sessions (list I may need to add it to)
    comp_subject_number = session.subject_number # Get both new session and subject number
    comp_session_number = session.session_number #   save them as comp_
    if (comp_session_number == new_session_number) and (comp_subject_number == new_subject_number): # If comp session and subject are equal to new subject and session
      should_add_session = False # In this case, that would mean that I should /not/ add the new session

  if should_add_session: # If I should add the session
    newSession = Session(new_subject_number, new_session_number) # Initialize the new session object with the new subject and session values
    list_of_sessions.append(newSession) # Append the new session to the list of sessions
  break
# Now list_of_sessions has a list of Session objects for each of the sessions

# Print out the list of the sessions in list_of_sessions:
# for session in list_of_sessions:
#   newStr = "Subject: " + str(session.subject_number) + ". Session: " + str(session.session_number)
#   print(newStr) 


In [97]:
# Code block created by Caleb
# Goal: Create a function that will take in an array of size [(length of file) x 6] and pad it with zeros for image size

# This function pads with image width - 1 rows of zeros
def padData(data, image_width):
  dim = [image_width - 1,6] # The dimensions of the zeros will be the width of the image and six wide
  zeroArray = np.zeros(dim) # Create an array with zeros at the beginning of the correct size
  return np.concatenate((zeroArray, data)) # Concatonate the two arrays together with the zeros at the beginning

# Print out the data for a small data array to see where the zeros go
data = np.array([[1, 2, 3, 4, 5, 6], [3, 4, 5, 6, 7, 5]])
paddedData = padData(data, 3) 
print(data)

[[1 2 3 4 5 6]
 [3 4 5 6 7 5]]


In [98]:
# Code block created by Caleb
# Goal: Step three of the whiteboard photo, so create "images" from padded data

# This function takes in the padded data (an array  [(image_width + 40*session lenght).x 6] ) and an integer of the width of an image
def createImages(paddedData, image_width):
  all_images = [] # Create an empty array that will eventually hold all the images for this session's data
  number_of_images = paddedData.shape[0] - image_width + 1  # The number of images I should create is padded_data - (image_width - 1)
  for i in range(number_of_images): # Loop through i for each of the images I need to create
    all_images.append(paddedData[i:(i+image_width)])  # Grab out the array from the padded data equal to the image_width and starting at i, and append it to all images
  return np.asarray(all_images) 


# This example uses the 'data' array from the code block above to pass into this function to see if it successfully creates two images
#   One should have two rows of zeros (because it gets all the padding) and the next should have one row of zeros
images = createImages(paddedData, 3)
print(images)

[[[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [1. 2. 3. 4. 5. 6.]]

 [[0. 0. 0. 0. 0. 0.]
  [1. 2. 3. 4. 5. 6.]
  [3. 4. 5. 6. 7. 5.]]]


In [99]:
# I will use the first session for this example:
example_session = list_of_sessions[0]

example_x_data = example_session.getXDataFromFile()
example_x_time = example_session.getXTimeFromFile()
#example_y_data = example_session.getYDataFromFile() # y data does not exist here
example_y_time = example_session.getYTimeFromFile()

# No need to extrapolate the ys 
#xample_extrapolated_y = extrapolatedYs(example_x_time, example_y_time, example_y_data)

# Now the x data and the extrapolate_y have the correct sizes, so y is the predictions we want
print("New y size, and then x data shape")

print(example_x_data.shape)
print("- - -")

# Now I will show the examples of transforming the x values into images. 
example_image_width = 60 # You can play around with the width of the image
example_x_padded = padData(example_x_data, example_image_width)
print("With padding:")
print(example_x_padded.shape)
print("- - -")

example_x_images = createImages(example_x_padded, example_image_width)
print("Shape of images, so should be back down to number of x values, with image_size x 6 secondary dimensions")
print(example_x_images.shape)
print("- - -")
print("Printing first images")
# print(example_x_images[0])
# print(example_x_images[1])

# So the data we're treating just like images for classification are:
# example_x_images          as the images
# example_extrapolated_y    as the labels

New y size, and then x data shape
(37991, 6)
- - -
With padding:
(38050, 6)
- - -
Shape of images, so should be back down to number of x values, with image_size x 6 secondary dimensions
(37991, 60, 6)
- - -
Printing first images


In [100]:


# Code block created by Caleb

def getImagesFromSession(session, image_width):
  image_width_number = int(image_width * 40) # This is the conversion rate between image_width in seconds and image_width the number of samples in an image

  x_data = session.getXDataFromFile() # Get all the data for that session
  x_time = session.getXTimeFromFile()
  y_time = session.getYTimeFromFile()

  
  x_padded = padData(x_data, image_width_number) # Pad the x values
  x_images = createImages(x_padded, image_width_number) # Then create images



  print("Done processing session!")


  return np.array(x_time), np.asarray(x_images), np.asarray(y_time)

#Yt_train = Yt_train.type(torch.LongTensor)
# This next line calls the function to get the data for the model with the parameters passed in.
# Remember that image width and set length are in seconds



In [101]:
our_transform = transforms.Compose([
    # Converting RGB [0,255] to Tensor [0,1]
    transforms.ToTensor(),
    ])

# def getYLabels(session, image_width, model):
#   x_time, x_images, y_time = getImagesFromSession(session, image_width)
#   model.eval()
#   test_data = CustomImageDataset(x_images, x_time, transform=our_transform) # x_time is just filler here 
#   test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(x_images))
#   _iter = iter(test_loader)
#   print(type(_iter))
#   data, index = next(_iter)
#   print(data.shape)
#   print(type(data))
#   predictions = model(data)
#   return x_time, x_images, y_time, predictions





def getYLabels(session, image_width, model): #TODO: Could there be a bug here? So our model is good but we arent spitting things out in the righ place?????
  x_time, x_images, y_time = getImagesFromSession(session, image_width)
  #print("X_Images Shape:", x_images.shape)
  predictions = []

  model.eval()
  test_data = CustomImageDataset(x_images, x_time, transform=our_transform) # x_time is just filler here 
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=20)
  for data, _target in test_loader:
  
    output = model(data)
    output_numpy =  output.detach().numpy()
    for out in output_numpy:
      predictions.append(out)

  return x_time, x_images, y_time, np.asarray(predictions)


  

In [102]:
def loadModelStateFromFile(model, filename):
  ''' Load model's state onto model passed in from a model file'''
  competition_path = "/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Models/" #path to data folder in drive
  model_file_path = os.path.join(competition_path, filename+".pt")  #joing path and adding .npy to filename passed in
  print(model_file_path)
  model.load_state_dict(torch.load(model_file_path, map_location=torch.device('cpu'))) # Should not need a return since the model will be modified through its object 


In [103]:
# Custom CNN
# Defining the CNN architecture
#Trenton's Net class
# class Net(nn.Module):
#   def __init__(self):
#     super(Net, self).__init__()
#     self.conv1 = nn.Conv2d(1, 6, 5, padding=2, padding_mode='zeros', dtype=float)
#    ## nn.MaxPool2d()
#     self.conv2 = nn.Conv2d(6, 16, 5, dtype=float)
#     self.fc1 = nn.Linear(16 * 56 * 2, 800, dtype=float)
#     self.fc2 = nn.Linear(800, 200, dtype=float) 
#     self.fc3 = nn.Linear(200, 50, dtype=float)
#     self.fc4 = nn.Linear(50, 4, dtype=float)
#   def forward(self, x):
#     x = F.relu(self.conv1(x))
#     x = F.relu(self.conv2(x))
#     x = x.view(-1, 16 * 56 * 2) # At this point the feature map is 5 x 5 x 16
#     x = F.relu(self.fc1(x))
#     x = F.relu(self.fc2(x))
#     x = F.relu(self.fc3(x))
#     x = self.fc4(x)
#     return x
# class Net(nn.Module):
#   def __init__(self):
#     super(Net, self).__init__()
#     self.conv1 = nn.Conv2d(1, 6, 5, padding=2, padding_mode='zeros', dtype=float)
#    ## nn.MaxPool2d()
#     self.conv2 = nn.Conv2d(6, 16, 5, dtype=float)
#     self.fc1 = nn.Linear(16 * 36 * 2, 800, dtype=float)
#     self.fc2 = nn.Linear(800, 200, dtype=float) 
#     self.fc3 = nn.Linear(200, 50, dtype=float)
#     self.fc4 = nn.Linear(50, 4, dtype=float)
#   def forward(self, x):
#     x = F.relu(self.conv1(x))
#     x = F.relu(self.conv2(x))
#     x = x.view(-1, 16 * 36 * 2) # At this point the feature map is 5 x 5 x 16
#     x = F.relu(self.fc1(x))
#     x = F.relu(self.fc2(x))
#     x = F.relu(self.fc3(x))
#     x = self.fc4(x)
#     return x

# class Net(nn.Module):
#   def __init__(self, ):
#     super(Net, self).__init__()
#     self.conv1 = nn.Conv2d(1, 4, 5, padding=2, padding_mode='zeros', dtype=float)
#    ## nn.MaxPool2d()
#     self.conv2 = nn.Conv2d(4, 8, 5, padding=2, padding_mode='zeros', dtype=float)
#     self.drop2 = nn.Dropout2d(0.1)
#     self.drop1 = nn.Dropout(0.1)
#     self.batchNorm1 = nn.BatchNorm2d(4, dtype = float)
#     self.batchNorm2 = nn.BatchNorm2d(8, dtype = float)
#     self.batchNorm3 = nn.BatchNorm1d(512, dtype = float)
#     self.batchNorm4 = nn.BatchNorm1d(128, dtype = float)
#     self.batchNorm5 = nn.BatchNorm1d(32, dtype = float)
#     self.fc1 = nn.Linear(8 * 40 * 6, 512, dtype=float)
#     self.fc2 = nn.Linear(512, 128, dtype=float) 
#     self.fc3 = nn.Linear(128, 32, dtype=float)
#     self.fc4 = nn.Linear(32, 4, dtype=float)
#   def forward(self, x):
#     x = self.drop2(F.relu(self.batchNorm1(self.conv1(x))))
#     x = self.drop2(F.relu(self.batchNorm2(self.conv2(x))))
#     x = x.view(-1, 8 * 40 * 6) # At this point the feature map is 5 x 5 x 16
#     x = self.drop1(F.relu(self.batchNorm3(self.fc1(x))))
#     x = self.drop1(F.relu(self.batchNorm4(self.fc2(x))))
#     x = self.drop1(F.relu(self.batchNorm5(self.fc3(x))))
#     x = self.fc4(x)
#     return x

# Nathan/Caleb's Randomization Working Net:
# Defining the CNN architecture
# class Net(nn.Module):
#   def __init__(self, ):
#     super(Net, self).__init__()
#     self.conv1 = nn.Conv2d(1, 6, 5, padding=2, padding_mode='zeros', dtype=float)
#    ## nn.MaxPool2d()
#     self.conv2 = nn.Conv2d(6, 16, 5, padding=2, padding_mode='zeros', dtype=float)
#     self.drop2 = nn.Dropout2d(0.1)
#     self.drop1 = nn.Dropout(0.1)
#     self.batchNorm1 = nn.BatchNorm2d(6, dtype = float)
#     self.batchNorm2 = nn.BatchNorm2d(16, dtype = float)
#     self.batchNorm3 = nn.BatchNorm1d(800, dtype = float)
#     self.batchNorm4 = nn.BatchNorm1d(200, dtype = float)
#     self.batchNorm5 = nn.BatchNorm1d(50, dtype = float)
#     self.fc1 = nn.Linear(16 * 60 * 6, 800, dtype=float)
#     self.fc2 = nn.Linear(800, 200, dtype=float) 
#     self.fc3 = nn.Linear(200, 50, dtype=float)
#     self.fc4 = nn.Linear(50, 4, dtype=float)
#   def forward(self, x):
#     x = self.drop2(F.relu(self.batchNorm1(self.conv1(x))))
#     x = self.drop2(F.relu(self.batchNorm2(self.conv2(x))))
#     x = x.view(-1, 16 * 60 * 6) # At this point the feature map is 5 x 5 x 16
#     x = self.drop1(F.relu(self.batchNorm3(self.fc1(x))))
#     x = self.drop1(F.relu(self.batchNorm4(self.fc2(x))))
#     x = self.drop1(F.relu(self.batchNorm5(self.fc3(x))))
#     x = self.fc4(x)
#     return x


# Nathan's 90+% model
class Net(nn.Module):
  def __init__(self, ):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 128, (30,6), dtype=float)
    self.drop2 = nn.Dropout2d(0.5)
    self.drop1 = nn.Dropout(0.2)
    self.maxPool = nn.MaxPool1d(1, 31)
    # self.batchNorm1 = nn.BatchNorm2d(128, dtype = float)
    # self.batchNorm2 = nn.BatchNorm2d(16, dtype = float)
    # self.batchNorm3 = nn.BatchNorm1d(800, dtype = float)
    # self.batchNorm4 = nn.BatchNorm1d(200, dtype = float)
    # self.batchNorm5 = nn.BatchNorm1d(50, dtype = float)
    self.fc1 = nn.Linear(128 * 31 * 1, 128, dtype=float)
    self.fc2 = nn.Linear(128, 64, dtype=float) 
    self.fc3 = nn.Linear(64, 32, dtype=float)
    self.fc4 = nn.Linear(32, 4, dtype=float)
  def forward(self, x):
    x = self.drop2(F.relu(self.conv1(x)))
    # x = x.view(-1, 128 * 30 * 6) 
    x = x.view(-1, 128 * 31 * 1) 
    # x= self.maxPool(x)
    x = self.drop1(F.relu(self.fc1(x)))
    x = self.drop1(F.relu(self.fc2(x)))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x



In [104]:
# test_session = list_of_sessions[0]
# test_model = Net()

# loadModelStateFromFile(test_model, "Test_Model")
# test_x_time, test_x_data, test_y_time, test_predictions = getYLabels(test_session, 1.5, test_model)


In [105]:
# print(test_predictions)

# print(test_predictions[0])


# print(test_predictions[100][0])
# print(test_predictions[100][1])
# print(test_predictions[100][2])
# print(test_predictions[100][3])


In [106]:

# # for test_pred in test_predictions:
# #   print(test_pred.shape)
# _, max_indexes = torch.max(test_predictions, 1)
# #max_indexes.append(max_index)






In [107]:
# print(max_indexes[1000])

# # for index in max_indexes:
# #   print(int(index))
# y_lined_up_with_x = []
# for index in max_indexes:
#   y_lined_up_with_x.append(int(index))

# y_lined_up_with_x = np.asarray(y_lined_up_with_x)

In [108]:
# "Downsample" Y labels to correpond to what needs to be submitted
def deExtrapolateYs(x_time, y_time, y_lined_up_with_x):
  correctY = [] #Empty list for the newly transformed y values
  x_time_array = np.asarray(x_time)
  for y_t in y_time: # Loop over every y_time value
    index = (np.abs(x_time_array - y_t)).argmin() # Get index of closeset x_time to the y_t you are currently looking for
    correctY.append(y_lined_up_with_x[index]) # Add the y value at that index to the transformed y's
  return np.asarray(correctY)


In [109]:
# actual_ys = deExtrapolateYs(test_x_time, test_y_time, y_lined_up_with_x)

In [110]:
# print(actual_ys.shape)

In [111]:
# print(actual_ys[0])

In [112]:
# pd.DataFrame(actual_ys).to_csv('first_y_perhaps')

In [113]:
# np.savetxt('/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Caleb Playing with It/first_y_save_txt', actual_ys)

In [114]:
# pd.DataFrame(actual_ys).to_csv('/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Caleb Playing with It/first_y_maybe')

In [115]:
# actual_ys.tofile('/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Caleb Playing with It/first_y_maybe', sep='\n')

In [116]:
def get_Y_values_from_session(session):
  # basicaly just getting data as numpy array 
  # need to make sure y_session data even exits, may be on session without data
  y_data_path = data_folder_path + "TrainingData/" + "subject_" + f"{session.subject_number:03d}" + "_" + f"{session.session_number:02d}" + "__y.csv" # Get the path to the x_data file 
  try:  
    y_data = np.genfromtxt(y_data_path, delimiter=',')  # Read the data in from the text file
  except:
    print(y_data_path, " was NOT FOUND")
    return None # returns none so we cna exit
  return y_data  # Return the data array
  

In [117]:
def extrapolatedYs(x_time, y_time, y):
  transformedY = [] #Empty list for the newly transformed y values
  y_time_array = np.asarray(y_time) # Create the y_time values as an array
  for x_t in x_time: # Loop over every x_time value
    index = (np.abs(y_time_array - x_t)).argmin() # I found this online to get the index of the closest value from the y_time arrays
    transformedY.append(y[index]) # Add the y value at that index to the transformed y's
  return np.asarray(transformedY)

In [118]:
 from sklearn.metrics import precision_recall_fscore_support 
 classes = [0, 1, 2, 3]

def get_session_accuracy_and_f1_score(indv_session, predcitions_y_values):
  session_true_y_values = get_Y_values_from_session(indv_session)
  session_upsampled_y_values = extrapolatedYs(session.getXTimeFromFile(), session.getYTimeFromFile(),  session_true_y_values)
  temp_precision_all, temp_recall_all, temp_fscore_all, _ = precision_recall_fscore_support(session_upsampled_y_values, predcitions_y_values, average="macro", labels=classes, zero_division=0, beta=1) 
  amount_correct = np.sum(predcitions_y_values == session_upsampled_y_values)

  # get amount of each number 
  amount_per_class = [0, 0, 0, 0]
  amount_guessed_correct_per_class = [0, 0, 0, 0]
  amount_guessed_per_class = [0, 0, 0 ,0]
  accuracy_per_class = [0, 0, 0, 0]
  guessed_vs_real_ratio = [0, 0, 0, 0]
  
  for index, true_value in enumerate(session_upsampled_y_values):
    amount_per_class[int(true_value)] +=1
    if(true_value == predcitions_y_values[index]):
      amount_guessed_correct_per_class[int(true_value)] += 1
  
  for label in classes:
    if amount_per_class[label] == 0:
        amount_per_class[label] = .000001
    accuracy_per_class[label] = amount_guessed_correct_per_class[label]/amount_per_class[label] # getting accuracy 
    amount_guessed_per_class[label] = np.sum(predcitions_y_values == label)
    guessed_vs_real_ratio[label] = amount_guessed_per_class[label]/ amount_per_class[label]



  accuracy = amount_correct/session_upsampled_y_values.shape[0]
  return accuracy, temp_fscore_all, accuracy_per_class, guessed_vs_real_ratio


def get_session_accuracy_and_f1_score_undextrapolated(indv_session, predcitions_y_values):
  session_true_y_values = get_Y_values_from_session(indv_session)
  session_upsampled_y_values = extrapolatedYs(session.getXTimeFromFile(), session.getYTimeFromFile(),  session_true_y_values)
  temp_precision_all, temp_recall_all, temp_fscore_all, _ = precision_recall_fscore_support(session_true_y_values, predcitions_y_values, average="macro", labels=classes, zero_division=0, beta=1) 
  amount_correct = np.sum(predcitions_y_values == session_true_y_values)

  # get amount of each number 
  amount_per_class = [0, 0, 0, 0]
  amount_guessed_correct_per_class = [0, 0, 0, 0]
  amount_guessed_per_class = [0, 0, 0 ,0]
  accuracy_per_class = [0, 0, 0, 0]
  guessed_vs_real_ratio = [0, 0, 0, 0]
  
  for index, true_value in enumerate(session_true_y_values):
    amount_per_class[int(true_value)] +=1
    if(true_value == predcitions_y_values[index]):
      amount_guessed_correct_per_class[int(true_value)] += 1
  
  for label in classes:
    if amount_per_class[label] == 0:
        amount_per_class[label] = .000001
    accuracy_per_class[label] = amount_guessed_correct_per_class[label]/amount_per_class[label] # getting accuracy 
    amount_guessed_per_class[label] = np.sum(predcitions_y_values == label)
    guessed_vs_real_ratio[label] = amount_guessed_per_class[label]/ amount_per_class[label]



  accuracy = amount_correct/session_true_y_values.shape[0]
  return accuracy, temp_fscore_all, accuracy_per_class, guessed_vs_real_ratio



In [120]:
model = Net()
loadModelStateFromFile(model, "report_model") # "Test_Model" for bad 77% model

one_session = []
#one_session.append(list_of_sessions[1])

for session in list_of_sessions:
  x_time, x_data, y_time, predictions = getYLabels(session, 1.5, model) # This is 1.5 for bad 77% model
  predictions = torch.from_numpy(predictions)
  _, max_indexes = torch.max(predictions, 1)

  y_lined_up_with_x = []
  for index in max_indexes:
    y_lined_up_with_x.append(int(index))

  y_lined_up_with_x = np.asarray(y_lined_up_with_x)

  y_values = deExtrapolateYs(x_time, y_time, y_lined_up_with_x)
  
  y_values.tofile(f'/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Data/Best_Submission/subject_{session.subject_number:03d}_{session.session_number:02d}__y.csv', sep='\n')
  #y_lined_up_with_x.tofile(f'/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Caleb Playing with It/NonDesampled/subject_{session.subject_number:03d}_{session.session_number:02d}__y.csv', sep='\n')

  # print out accuracy and f1score
  if TRAINING:  
    if False:
      total_accuracy, total_f1score, accuracy_per_class, guessed_vs_real_ratio = get_session_accuracy_and_f1_score(session, y_lined_up_with_x) # USING Y_LINED_UP_WITH_X
    else:
      total_accuracy, total_f1score, accuracy_per_class, guessed_vs_real_ratio = get_session_accuracy_and_f1_score_undextrapolated(session, y_lined_up_with_x)
    print("Subject: ", session.subject_number, " Session: ", session.session_number)
    print("Total Accuracy: ", total_accuracy)
    print("Total F1Score!: ", total_f1score)
    print("Accuracy Per Class: ", accuracy_per_class)
    print("Guessed/Real: ", guessed_vs_real_ratio, " Ideal = 1, Overguessed > 1, Underguessed < 1")

  else:
    pass


/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Models/report_model.pt
Done processing session!
/content/drive/Shareddrives/Neural Nets/Competition/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_009_01__y.csv  was NOT FOUND


TypeError: ignored

In [ ]:
session_y_value = get_Y_vlaues_from_session(session)

In [ ]:
print(y_values.shape)
print(session_y_value.shape)

In [ ]:
print(accuracy)

In [ ]:
 from sklearn.metrics import precision_recall_fscore_support 
 
 temp_precision_all, temp_recall_all, temp_fscore_all, _ = precision_recall_fscore_support(session_y_value, y_values, average="macro", labels=classes, zero_division=0, beta=1) 

In [ ]:
print(temp_fscore_all)

In [ ]:
total_accuracy, total_f1score = get_session_accuracy_and_f1_score(session, y_values)
print("Total Accuracy: ", total_accuracy)
print("Total F1Score:  ", total_f1score)